In [ ]:
# %pip install --upgrade gradio

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from Fourier_Transformer import LinearTransformer, create_inout_sequences

c:\Users\19495\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the model and prepare the user interface.

In [3]:
model = LinearTransformer(feature_size=1, num_layers=2, d_model=64, num_heads=8, d_ff=2048, dropout=0.1)
model.load_state_dict(torch.load('transformer_fourier.pth'))
model.eval()

LinearTransformer(
  (fourier_transform): FourierTransform()
  (positional_encoder): Embedding(1000, 64)
  (encoder_layers): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (fourier_transform): FourierTransform()
      (feed_forward): PositionwiseFeedforward(
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
      )
      (layernorm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-1): 2 x TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
      )
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64,

In [4]:
def predict_price(file_obj):
    # Read the CSV file
    df = pd.read_csv(file_obj)
    
    # Processing the data similar to your provided code
    df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
    df.dropna(inplace=True)  # Remove NaNs
    
    # Initialize scaler and scale the log returns
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df['log_return'] = scaler.fit_transform(df['log_return'].values.reshape(-1, 1))
    new_input_data = df['log_return']

    # Convert to Tensor
    new_input_tensor = torch.FloatTensor(new_input_data.to_numpy()).view(-1)
    
    # Create sequences
    seq_length = 10  # Example sequence length, adjust based on your training
    new_sequences = create_inout_sequences(new_input_tensor, seq_length)
    new_sequences = [seq[0] for seq in new_sequences]  # Get only the sequences

    # DataLoader for predictions
    predict_loader = DataLoader(new_sequences, batch_size=64, shuffle=False)

    # Prediction
    model.eval()  # Ensure the model is in evaluation mode
    predictions = []
    with torch.no_grad():
        for sequences in predict_loader:
            sequences = sequences.view(sequences.shape[0], seq_length, 1)
            output = model(sequences)
            predicted_values = output[:, -1]
            predictions.extend(predicted_values.numpy())

    # Inverse transform predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    # Calculate predicted price
    last_known_price = df['Close'].iloc[-1]
    predicted_price = last_known_price * np.exp(predictions[0][0])

    return f"Last Actual Price: {last_known_price}", f"Predicted Price: {predicted_price}"

In [5]:
# Define the Gradio interface
interface = gr.Interface(
    fn=predict_price,
    inputs=gr.File(label="Upload CSV File"),
    outputs=[gr.Textbox(label="Last Actual Price"), gr.Textbox(label="Predicted Price")],
    title="Stock Price Prediction",
    description="Upload a CSV file with 'Close' prices to predict the next price in the sequence."
)

In [6]:
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
